In [ ]:
pip install pyxlsb
pip install sqlalchemy psycopg2-binary


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import pandas as pd
import numpy as np
import re
import unicodedata
import warnings
import time
from datetime import datetime
import sys

# --------- Réduire le bruit ---------
warnings.filterwarnings("ignore", message="Could not infer format.*", category=UserWarning)

# ================== PARAMS ================== #
ROOT_DIR = Path("fic")
SHOW_SAMPLES = True

# >>> Perf toggles
QUICK_MODE = True              # True = plus rapide (moins d'inférence)
INFER_TYPES = True             # Inférence activée
VERBOSE = True                 # logs

# Limiteurs
MAX_FILES = None
MAX_SHEETS = None
SCAN_MAX_ROWS = 8000
MAX_TABLES_PER_SHEET = 2

# Heuristiques
MIN_COLS = 2
MIN_CONSEC_ROWS = 3 if QUICK_MODE else 5
STOP_EMPTY_RUN = 3 if QUICK_MODE else 5
HEADER_SCAN_DEPTH = 3 if QUICK_MODE else 6

# Segmentation verticale
COL_DENSITY_THRESHOLD = 0.12
MIN_COL_RUN = 2
ALLOW_SMALL_GAPS = 1
MAX_SEGMENTS_PER_SHEET = None

# Filtrage colonnes “génériques”
MIN_NON_NULL_RATIO = 0.05
MIN_NON_NULL_ABS   = 2
GENERIC_COL_RE = re.compile(r"^col(_\d+)?$", re.I)

# ================== INFÉRENCE: CONSTANTES ================== #
DATE_NAME_HINTS = ("date","dt_","_dt","attribution","retrait","month","mois","jour","day","time","heure")
TIME_TOKEN_RE = re.compile(r"\d{1,2}:\d{2}(?::\d{2})?")
DATE_TOKEN_RE = re.compile(
    r"[/\-.]|(?:jan|feb|mar|apr|mai|may|jun|jul|aug|sep|oct|nov|dec|"
    r"janv|févr|fevr|avr|juil|sept|oct|nov|d[ée]c)",
    re.I
)

# Monnaies: regex strictes
# (fix: groupe NON capturant pour .str.contains)
CURRENCY_DETECT_VALUE_RE = re.compile(r"(?:€|\$|usd|eur|dhs?|mad|£|gbp)", re.I)
CURRENCY_PARSE_RE = re.compile(
    r"(?P<neg>\()?\s*(?P<cur>€|\$|usd|eur|dhs?|mad|£|gbp)?\s*"
    r"(?P<num>[+-]?\s?(?:\d{1,3}(?:[ .,\u00A0]\d{3})+|\d+)(?:[.,]\d+)?)(?(neg)\))",
    re.I
)
CURRENCY_NAME_HINTS = ("amount","montant","price","prix","total","ttc","ht","paid","due","debit","credit")

BOOL_TRUE  = {"true","vrai","oui","y","o","yes","1"}
BOOL_FALSE = {"false","faux","non","n","no","0"}

# Seuils prudents
RATIO_STRICT_BOOL = 0.98     # bool seulement si quasi-pur
RATIO_NUM         = 0.85     # numérique si majorité claire
RATIO_DATE_HARD   = 0.70
RATIO_DATE_SOFT   = 0.30

# >>> Exports (optionnels)
EXPORT_DIR = Path("out_tables")
EXPORT_AS = None              # "csv" | "parquet" | None
EXPORT_PG_DDL = True          # écrire un .sql (CREATE TABLE) par table

# -------------------- logging utils -------------------- #
def nowstr() -> str:
    return datetime.now().strftime("%H:%M:%S")

def log(msg: str) -> None:
    if VERBOSE:
        print(f"[{nowstr()}] {msg}", flush=True)

def step_time(prev=None):
    t = time.perf_counter()
    if prev is None:
        return t, 0.0
    return t, (t - prev)

# -------------------- utils noms -------------------- #
def strip_accents_lower(s: str) -> str:
    if s is None or pd.isna(s):
        return ""
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

def snake_id(s: str) -> str:
    s = strip_accents_lower(s)
    s = re.sub(r"[\s\.\-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or "sheet"

def is_generic_colname(name: str) -> bool:
    return bool(GENERIC_COL_RE.fullmatch(name or ""))

# -------------------- Helpers parsing -------------------- #
def looks_like_code(values: pd.Series) -> bool:
    """Codes avec zéros en tête / longueur quasi fixe (4-10) -> garder texte."""
    s = values.astype("string").dropna().str.strip()
    if len(s) == 0:
        return False
    digit_only = s.str.fullmatch(r"\d+")
    if (s.str.startswith("0") & digit_only).any():
        return True
    lengths = s.where(digit_only).str.len().dropna()
    if len(lengths) > 0:
        mode_len = lengths.mode().iloc[0]
        if (lengths.eq(mode_len).mean() >= 0.80) and (4 <= mode_len <= 10):
            return True
    return False

def parse_number_like(s: pd.Series) -> pd.Series:
    """Normalise nombres FR/US."""
    x = s.astype("string")
    x = x.str.replace("\u00A0", " ", regex=False)
    x = x.str.replace(" ", "", regex=False)
    # retire . de milliers: 1.234,56 -> 1234,56 (mais ne touche pas aux décimales US .123)
    x = x.str.replace(r"(?<=\d)\.(?=\d{3}(?:\D|$))", "", regex=True)
    x = x.str.replace(",", ".", regex=False)
    return pd.to_numeric(x, errors="coerce")

def parse_currency(series: pd.Series, col_name: str) -> tuple[pd.Series, bool]:
    """Détecte monnaie seulement si symbole/code présent dans les valeurs OU si nom le suggère."""
    s = series.astype("string")
    name_hint = any(h in strip_accents_lower(col_name or "") for h in CURRENCY_NAME_HINTS)
    has_symbol = s.str.contains(CURRENCY_DETECT_VALUE_RE, na=False)
    if not name_hint and has_symbol.mean() < 0.5:
        return pd.Series(pd.NA, index=series.index, dtype="Float64"), False

    def _one(v):
        if v is None or (isinstance(v, float) and np.isnan(v)):
            return np.nan
        txt = str(v)
        m = CURRENCY_PARSE_RE.search(txt)
        if not m:
            return parse_number_like(pd.Series([txt])).iloc[0]
        num = m.group("num") or ""
        neg = bool(m.group("neg"))
        val = parse_number_like(pd.Series([num])).iloc[0]
        if pd.isna(val):
            return np.nan
        return -val if neg or ("(" in txt and ")" in txt) else val

    out = s.map(_one).astype("Float64")
    return out, True

def parse_percent(s: pd.Series, col_name: str) -> tuple[pd.Series, bool]:
    """Pourcentages '12,5%' ou '12.5%' ; si '%' absent, exige nom de col type rate/ratio/taux."""
    raw = s.astype("string")
    has_pct = raw.fillna("").str.contains(r"%")
    name_hint = any(h in strip_accents_lower(col_name or "") for h in ("rate", "ratio", "taux", "pourcent", "pct"))
    if has_pct.mean() >= 0.6 or name_hint:
        x = raw.str.replace("%", "", regex=False)
        vals = parse_number_like(x).astype("Float64")
        return vals, True
    return pd.Series(pd.NA, index=s.index, dtype="Float64"), False

def try_parse_datetime(series: pd.Series, favor_day_first=True):
    """Essaie datetime (heure) puis date, double passe (jour/mois)."""
    s = series.astype("string")
    time_ratio = s.str.contains(TIME_TOKEN_RE, na=False).mean() if len(s) > 0 else 0.0
    dt1 = pd.to_datetime(s, errors="coerce", dayfirst=favor_day_first)
    r1 = dt1.notna().mean() if len(s.dropna()) > 0 else 0.0
    if r1 < RATIO_DATE_HARD:
        dt2 = pd.to_datetime(s, errors="coerce", dayfirst=not favor_day_first)
        r2 = dt2.notna().mean() if len(s.dropna()) > 0 else 0.0
        dt, rr = (dt2, r2) if r2 > r1 else (dt1, r1)
    else:
        dt, rr = dt1, r1
    if rr >= RATIO_DATE_HARD or (rr >= RATIO_DATE_SOFT and any(h in strip_accents_lower(series.name or "") for h in DATE_NAME_HINTS)):
        if time_ratio >= 0.20 or (dt.dt.time.astype(str) != "00:00:00").mean() > 0.20:
            return dt, "DATETIME"
        return dt.dt.normalize(), "DATE"
    return pd.Series(pd.NaT, index=series.index), None

# ---------- Lecture Excel/CSV ----------
def safe_read_excel_all_sheets(path: Path):
    ext = path.suffix.lower()
    if ext in (".xlsx", ".xlsm"):
        engine = "openpyxl"
    elif ext == ".xlsb":
        engine = "pyxlsb"
    else:
        raise ValueError(f"Extension Excel non gérée: {ext}")
    log(f"  -> Ouverture Excel ({engine})")
    t0, _ = step_time()
    xls = pd.ExcelFile(path, engine=engine)
    sheets = xls.sheet_names[:(MAX_SHEETS or len(xls.sheet_names))]
    out = []
    nrows = SCAN_MAX_ROWS if SCAN_MAX_ROWS is not None else None
    for i, sheet in enumerate(sheets, 1):
        ti, _ = step_time()
        log(f"    .. Lecture feuille {i}/{len(sheets)}: '{sheet}' nrows={nrows or 'ALL'}")
        df = pd.read_excel(xls, sheet_name=sheet, header=None, engine=engine, nrows=nrows)
        _, dt = step_time(ti)
        log(f"       -> taille: {df.shape[0]}x{df.shape[1]} (en {dt:.2f}s)")
        out.append((sheet, df))
    _, dt_total = step_time(t0)
    log(f"  -> Excel chargé en {dt_total:.2f}s")
    return out

COMMON_SEPS = [",", ";", "\t", "|"]
COMMON_ENCODINGS = ["utf-8-sig", "utf-8", "cp1252", "latin-1"]

def guess_sep(first_line: str):
    counts = {sep: first_line.count(sep) for sep in COMMON_SEPS}
    sep = max(counts, key=counts.get)
    return sep if counts[sep] > 0 else ","

def read_csv_fast(path: Path):
    t0, _ = step_time()
    log("  -> Détection encodage/séparateur (rapide)")
    enc_used = None
    sep_used = ","
    tried = []
    head = ""
    for enc in COMMON_ENCODINGS:
        try:
            tried.append(enc)
            with open(path, "r", encoding=enc, errors="strict") as f:
                head = f.readline()
            enc_used = enc
            break
        except UnicodeDecodeError:
            continue
        except Exception:
            enc_used = "latin-1"
            break
    if enc_used is None:
        enc_used = "latin-1"
    if not head:
        with open(path, "r", encoding=enc_used, errors="replace") as f:
            head = f.readline()
    sep_used = guess_sep(head)
    log(f"     -> encodage: {enc_used} (essais={tried}); sep='{sep_used}'")
    nrows = SCAN_MAX_ROWS if SCAN_MAX_ROWS is not None else None
    log(f"  -> Lecture CSV nrows={nrows or 'ALL'}")
    df = pd.read_csv(path, sep=sep_used, encoding=enc_used, header=None, nrows=nrows)
    _, dt = step_time(t0)
    log(f"     -> taille: {df.shape[0]}x{df.shape[1]} (en {dt:.2f}s)")
    return df

# ---------- Pré-traitements & détection ----------
def ensure_range_columns(df: pd.DataFrame):
    if list(df.columns) != list(range(df.shape[1])):
        df = df.copy()
        df.columns = list(range(df.shape[1]))
    return df

def detect_blocks(df: pd.DataFrame):
    t0, _ = step_time()
    df = ensure_range_columns(df)
    if df.empty:
        log("    .. Aucun bloc (df vide)")
        return []
    row_nnz = df.notna().sum(axis=1).to_numpy()
    tab = row_nnz >= MIN_COLS
    blocks = []
    if tab.any():
        padded = np.r_[False, tab, False]
        starts = np.where((~padded[:-1]) & (padded[1:]))[0]
        ends = np.where((padded[:-1]) & (~padded[1:]))[0] - 1
        for s, e in zip(starts, ends):
            if (e - s + 1) >= MIN_CONSEC_ROWS:
                blocks.append((df.index[s], df.index[e]))
                if QUICK_MODE and MAX_TABLES_PER_SHEET and len(blocks) >= MAX_TABLES_PER_SHEET:
                    break
    _, dt = step_time(t0)
    log(f"    .. Blocs tabulaires détectés (vertical/lignes): {len(blocks)} (en {dt:.2f}s)")
    return blocks

def split_by_vertical_gaps(df: pd.DataFrame):
    if df.empty or df.shape[1] <= 1:
        return [(df, 0, max(df.shape[1]-1, 0))] if df.shape[1] else []
    dens = df.notna().mean(axis=0).to_numpy()
    used = dens >= COL_DENSITY_THRESHOLD
    if ALLOW_SMALL_GAPS > 0:
        n = len(used)
        i = 0
        while i < n:
            if not used[i]:
                j = i
                while j < n and not used[j]:
                    j += 1
                gap_len = j - i
                left_used = (i - 1 >= 0 and used[i - 1])
                right_used = (j < n and used[j])
                if left_used and right_used and gap_len <= ALLOW_SMALL_GAPS:
                    used[i:j] = True
                i = j
            else:
                i += 1
    segments = []
    padded = np.r_[False, used, False]
    starts = np.where((~padded[:-1]) & (padded[1:]))[0]
    ends = np.where((padded[:-1]) & (~padded[1:]))[0] - 1
    for s, e in zip(starts, ends):
        if (e - s + 1) >= MIN_COL_RUN:
            sub = df.iloc[:, s:e + 1]
            segments.append((sub, s, e))
    return segments if segments else [(df, 0, df.shape[1] - 1)]

# ---------- Colonnes ----------
def choose_header_row(block: pd.DataFrame):
    best_idx, best_score = None, -1
    limit = min(len(block), HEADER_SCAN_DEPTH)
    for i in range(limit):
        row = block.iloc[i]
        vals = row.tolist()
        non_empty = sum(pd.notna(v) and str(v).strip() != "" for v in vals)
        texty = 0
        for v in vals:
            s = str(v).strip() if pd.notna(v) else ""
            if s and not s.lower().startswith("unnamed") and re.search(r"[A-Za-zÀ-ÿ]", s):
                texty += 1
        score = non_empty * 2 + texty
        if score > best_score:
            best_score, best_idx = score, i
    return best_idx or 0

def clean_columns(vals):
    cols, seen = [], {}
    for v in vals:
        s = strip_accents_lower(v).replace("\n", " ")
        s = re.sub(r"\s+", " ", s).strip(" -_")
        if not s or s.startswith("unnamed"):
            s = "col"
        s = re.sub(r"[^a-z0-9_ ]", "", s)
        s = re.sub(r"\s+", "_", s).strip("_") or "col"
        if s in seen:
            seen[s] += 1
            s = f"{s}_{seen[s]}"
        else:
            seen[s] = 1
        cols.append(s)
    return cols

def prune_columns(df: pd.DataFrame, header_keep: set[str]) -> pd.DataFrame:
    if df.empty:
        return df
    keep = list(header_keep)
    n = len(df)
    for c in df.columns:
        if c in header_keep:
            continue
        if not is_generic_colname(c):
            keep.append(c)
            continue
        nnz = df[c].notna().sum()
        if nnz >= max(MIN_NON_NULL_ABS, int(n * MIN_NON_NULL_RATIO)):
            keep.append(c)
    keep_ordered = [c for c in df.columns if c in keep]
    return df[keep_ordered].copy()

# ---------- Inférence de types ----------
def infer_and_cast_column(s: pd.Series, col_name: str) -> tuple[pd.Series, str]:
    """
    Retourne (Serie castée, type_str)
    type_str ∈ {"DATE","DATETIME","INT","FLOAT","CURRENCY","PERCENT","BOOL","CODE","STRING"}
    """
    if not INFER_TYPES:
        return s.astype("string"), "STRING"

    s_obj = s.astype("string")
    non_empty = s_obj.dropna().astype(str).str.strip()
    if len(non_empty) == 0:
        return s_obj.astype("string"), "STRING"

    # 0) codes
    if looks_like_code(non_empty):
        return s_obj.astype("string"), "CODE"

    # 1) dates/datetimes
    dt_vals, dt_tag = try_parse_datetime(s_obj)
    if dt_tag is not None:
        return (dt_vals, dt_tag)

    # 2) bool : seulement si quasi pur ET pas numérique dominant
    uniq_norm = pd.Series(non_empty).map(strip_accents_lower).dropna().unique().tolist()
    if 1 <= len(set(uniq_norm)) <= 3:
        mapped = s_obj.map(strip_accents_lower)
        def map_bool(x):
            if x in BOOL_TRUE:
                return True
            if x in BOOL_FALSE:
                return False
            return pd.NA
        mb = mapped.map(map_bool)
        frac_bool = mb.notna().sum() / max(1, mapped.notna().sum())
        # si la colonne est massivement numérique, privilégier le nombre
        as_num_probe = parse_number_like(s_obj)
        frac_num = as_num_probe.notna().sum() / max(1, s_obj.notna().sum())
        if frac_bool >= RATIO_STRICT_BOOL and frac_num < 0.9:
            return mb.astype("boolean"), "BOOL"

    # 3) currency stricte (symbole/code dans valeurs ou nom parlant)
    cur_vals, is_cur = parse_currency(s_obj, col_name)
    if is_cur:
        success = cur_vals.notna().mean() if s_obj.notna().mean() > 0 else 0.0
        if success >= RATIO_NUM:
            return cur_vals, "CURRENCY"

    # 4) percent
    pct_vals, is_pct = parse_percent(s_obj, col_name)
    if is_pct:
        success = pct_vals.notna().mean() if s_obj.notna().mean() > 0 else 0.0
        if success >= 0.70:
            return pct_vals, "PERCENT"

    # 5) numérique (FR/US)
    as_num = parse_number_like(s_obj)
    num_ratio = as_num.notna().sum() / max(1, s_obj.notna().sum())
    if num_ratio >= RATIO_NUM:
        as_int = as_num.dropna()
        if len(as_int) == 0:
            return as_num.astype("Float64"), "FLOAT"
        if (as_int % 1 == 0).all():
            return as_num.astype("Int64"), "INT"
        else:
            return as_num.astype("Float64"), "FLOAT"

    # 6) texte
    return s_obj.astype("string"), "STRING"

def infer_types_df(df: pd.DataFrame):
    t0, _ = step_time()
    out = df.copy()
    schema = {}
    for c in out.columns:
        out[c], t = infer_and_cast_column(out[c], c)
        schema[c] = t
    _, dt = step_time(t0)
    log(f"    .. Inférence types: {len(out.columns)} colonnes (en {dt:.2f}s, activée={INFER_TYPES})")
    return out, schema

def show_schema(df: pd.DataFrame, schema: dict):
    print("\n=== schema ===", flush=True)
    for c in df.columns:
        t = schema.get(c, str(df[c].dtype)).upper()
        print(f"{c}: {t}", flush=True)

# ---------- Mapping types -> PostgreSQL ----------
def pg_type_for(tag: str) -> str:
    tag = (tag or "STRING").upper()
    if tag == "DATE":
        return "date"
    if tag == "DATETIME":
        return "timestamp without time zone"
    if tag == "INT":
        return "integer"
    if tag in ("FLOAT", "PERCENT", "CURRENCY"):
        return "double precision"
    if tag == "BOOL":
        return "boolean"
    # CODE/STRING
    return "text"

def write_pg_ddl(table_name: str, df: pd.DataFrame, schema: dict):
    if not EXPORT_PG_DDL:
        return
    EXPORT_DIR.mkdir(parents=True, exist_ok=True)
    pg_table = snake_id(table_name)
    lines = []
    for c in df.columns:
        col = snake_id(str(c))
        pgt = pg_type_for(schema.get(c, "STRING"))
        lines.append(f'"{col}" {pgt}')
    ddl = f'CREATE TABLE "{pg_table}" (\n  ' + ",\n  ".join(lines) + "\n);"
    (EXPORT_DIR / f"{pg_table}.sql").write_text(ddl, encoding="utf-8")

# ---------- Extraction d'une table ----------
def carve_table_from_block(df_block: pd.DataFrame):
    if df_block.empty:
        return None
    # header
    h_rel = choose_header_row(df_block)
    cols = clean_columns(df_block.iloc[h_rel].tolist())
    data = df_block.iloc[h_rel + 1:].copy()
    if data.empty:
        return None
    data.columns = cols

    # couper après trop de lignes vides consécutives
    empty_run = 0
    cut_idx = data.index[-1]
    for idx in data.index:
        if data.loc[idx].isna().all():
            empty_run += 1
            if empty_run >= STOP_EMPTY_RUN:
                cut_idx = max(data.index[0], idx - STOP_EMPTY_RUN)
                break
        else:
            empty_run = 0
    data = data.loc[:cut_idx]
    data = data[data.notna().sum(axis=1) >= MIN_COLS]
    if data.empty:
        return None

    header_keep = {c for c in cols if not is_generic_colname(c)}
    data = prune_columns(data, header_keep)
    if data.empty:
        return None

    data, schema = infer_types_df(data)

    # mise en forme ISO pour PostgreSQL
    for c, t in schema.items():
        if t == "DATE":
            try:
                data[c] = pd.to_datetime(data[c], errors="coerce").dt.strftime("%Y-%m-%d")
            except Exception:
                pass
        elif t == "DATETIME":
            try:
                data[c] = pd.to_datetime(data[c], errors="coerce").dt.strftime("%Y-%m-%d %H:%M:%S")
            except Exception:
                pass
        elif t == "BOOL":
            data[c] = data[c].map(lambda x: True if x is True else (False if x is False else pd.NA))

    return data.reset_index(drop=True), schema

# ---------- Trouver tables dans une FEUILLE ----------
def find_tables_in_sheet(df_raw: pd.DataFrame):
    print("  -> Détection des blocs tabulaires…", flush=True)
    df_raw = ensure_range_columns(df_raw)
    if SCAN_MAX_ROWS is not None and len(df_raw) > SCAN_MAX_ROWS:
        df_raw = df_raw.iloc[:SCAN_MAX_ROWS, :]
        log(f"     (tronqué à {SCAN_MAX_ROWS} lignes)")

    segments = split_by_vertical_gaps(df_raw)
    if MAX_SEGMENTS_PER_SHEET:
        segments = segments[:MAX_SEGMENTS_PER_SHEET]
    log(f"  -> Segments verticaux: {len(segments)}")

    tables = []
    table_count = 0
    for seg_idx, (seg_df, c0, c1) in enumerate(segments, 1):
        log(f"  -> Segment {seg_idx}: cols {c0}..{c1} (shape {seg_df.shape[0]}x{seg_df.shape[1]})")
        if seg_df.empty:
            continue
        blocks = detect_blocks(seg_df)
        for k, (start, end) in enumerate(blocks, 1):
            log(f"     -> Carve table {k}/{len(blocks)} dans segment {seg_idx} (rows {start}..{end})")
            block = seg_df.loc[start:end, :]
            carved = carve_table_from_block(block)
            if carved is None:
                log("        .. ignoré (vide après carve)")
                continue
            table, schema = carved
            if table is not None and table.shape[1] >= MIN_COLS and table.shape[0] >= 1:
                tables.append((seg_idx, k, table, schema))
                table_count += 1
                log(f"        .. table retenue: {table.shape[0]}x{table.shape[1]}")
                if QUICK_MODE and MAX_TABLES_PER_SHEET and table_count >= MAX_TABLES_PER_SHEET:
                    log("        .. limite de tables atteinte (mode rapide)")
                    return tables
    log(f"  -> Tables retenues sur la feuille: {len(tables)}")
    return tables

# ---------- Par fichier ----------
def process_file(path: Path):
    results = []
    ext = path.suffix.lower()
    if ext in (".xlsx", ".xlsm", ".xlsb"):
        for sheet, df_raw in safe_read_excel_all_sheets(path):
            log(f"-- Feuille: {sheet} | taille {df_raw.shape[0]}x{df_raw.shape[1]}")
            for (seg_i, blk_i, t, sc) in find_tables_in_sheet(df_raw):
                results.append((sheet, seg_i, blk_i, t, sc))
    elif ext == ".csv":
        df_raw = read_csv_fast(path)
        log(f"-- CSV lu | taille {df_raw.shape[0]}x{df_raw.shape[1]}")
        for (seg_i, blk_i, t, sc) in find_tables_in_sheet(df_raw):
            results.append((None, seg_i, blk_i, t, sc))
    else:
        raise ValueError(f"Extension non gérée: {ext}")
    return results

def export_table(df: pd.DataFrame, name: str, schema: dict):
    if EXPORT_AS is None and not EXPORT_PG_DDL:
        return
    EXPORT_DIR.mkdir(parents=True, exist_ok=True)
    base = snake_id(name)
    if EXPORT_AS == "csv":
        # format compatible PG: séparateur virgule, décimales en '.', bool true/false
        df.to_csv(EXPORT_DIR / f"{base}.csv", index=False)
    elif EXPORT_AS == "parquet":
        df.to_parquet(EXPORT_DIR / f"{base}.parquet", index=False)
    if EXPORT_PG_DDL:
        write_pg_ddl(base, df, schema)

# ---------- main ----------
def main():
    print("=== data header: folder info ===", flush=True)
    print(f"path: {ROOT_DIR.resolve()}", flush=True)

    files = []
    for pat in ("*.xlsx", "*.xlsm", "*.xlsb", "*.csv"):
        files += [p for p in ROOT_DIR.rglob(pat) if p.is_file() and not p.name.startswith("~$")]
    files = sorted(files)
    if MAX_FILES:
        files = files[:MAX_FILES]
    print(f"files_found: {len(files)}", flush=True)

    print("\n=== config ===", flush=True)
    print(f"QUICK_MODE={QUICK_MODE} | INFER_TYPES={INFER_TYPES} | VERBOSE={VERBOSE}", flush=True)
    print(f"SCAN_MAX_ROWS={SCAN_MAX_ROWS} | MAX_FILES={MAX_FILES} | MAX_SHEETS={MAX_SHEETS} | MAX_TABLES_PER_SHEET={MAX_TABLES_PER_SHEET}", flush=True)
    print(f"COL_DENSITY_THRESHOLD={COL_DENSITY_THRESHOLD} | MIN_COL_RUN={MIN_COL_RUN} | ALLOW_SMALL_GAPS={ALLOW_SMALL_GAPS} | MAX_SEGMENTS_PER_SHEET={MAX_SEGMENTS_PER_SHEET}", flush=True)
    print(f"EXPORT_AS={EXPORT_AS} | EXPORT_DIR={EXPORT_DIR} | EXPORT_PG_DDL={EXPORT_PG_DDL}", flush=True)

    if not files:
        print("[warn] Aucun fichier trouvé.", flush=True)
        return

    total_tables = 0
    total_sheets = 0

    for f in files:
        print("\n=== file ===", flush=True)
        print(f"{f.name}  ({f.resolve()})", flush=True)
        t0 = time.perf_counter()
        try:
            tables = process_file(f)
        except Exception as e:
            print(f"[error] Lecture échouée pour {f.name}: {e}", flush=True)
            continue

        sheets_in_file = len({s for (s, *_rest) in tables if s is not None})
        total_sheets += sheets_in_file
        total_tables += len(tables)

        if not tables:
            print("[info] Aucune table détectée", flush=True)
        else:
            file_id = snake_id(f.stem)
            for (sheet, seg_i, blk_i, df, schema) in tables:
                sheet_id = snake_id(sheet or "sheet")
                table_idx = f"{seg_i}_{blk_i}"
                table_name = f"{file_id}.{sheet_id}.table_{table_idx}"
                print(f"\n--- table detected ---", flush=True)
                print(f"name: {table_name}", flush=True)
                print(f"rows: {len(df)} | cols: {df.shape[1]}", flush=True)
                print("columns:", ", ".join(map(str, df.columns.tolist())), flush=True)
                show_schema(df, schema)
                export_table(df, table_name, schema)
                if SHOW_SAMPLES:
                    with pd.option_context("display.max_columns", 80, "display.width", 200):
                        print("\n=== sample (top 8) ===", flush=True)
                        print(df.head(8), flush=True)

        t1 = time.perf_counter()
        print(f"[info] Temps fichier: {t1 - t0:.2f}s", flush=True)

    print("\n=== done ===", flush=True)
    print(f"Tables: {total_tables} | Feuilles (approx): {total_sheets} | Fichiers: {len(files)}", flush=True)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n[info] Interrompu par l'utilisateur.", flush=True)
        sys.exit(130)


# Connexion BDD

In [2]:
%pip install -q "sqlalchemy>=2" psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [1]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import pandas as pd

PG_HOST = "dpg-d3jq6apr0fns738f81i0-a.frankfurt-postgres.render.com"
PG_PORT = 5432
PG_DB   = "aerotec_datawarehouse"
PG_USER = "aerotec_datawarehouse_user"
PG_PASS = "LHTYZJ3aUDI8IeylbA1SZs9M9TsKQ4To"

conn_str = (
    f"postgresql+psycopg2://{PG_USER}:{quote_plus(PG_PASS)}@{PG_HOST}:{PG_PORT}/{PG_DB}"
)
engine = create_engine(conn_str, connect_args={"sslmode": "require"}, pool_pre_ping=True)

with engine.connect() as conn:
    print("OK, connecté.")
    print(conn.execute(text("SELECT version();")).scalar())

OK, connecté.
PostgreSQL 17.6 (Debian 17.6-1.pgdg12+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit


 # CAS 1

In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata, warnings

FILE_PATH = r"C:\globasoft\aerotech\fic\fichier qualité des trigrammes.xlsx"
SHOW_SAMPLE = True

# --- utils courts ---
def strip_accents_lower(s):
    if s is None or (isinstance(s, float) and pd.isna(s)): return ""
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

def snake_id(s):
    s = strip_accents_lower(s)
    s = re.sub(r"[\s\.\-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "_", s)
    return re.sub(r"_+", "_", s).strip("_") or "col"

def parse_number_like(s: pd.Series) -> pd.Series:
    x = s.astype("string").str.replace("\u00A0"," ", regex=False).str.replace(" ","", regex=False)
    x = x.str.replace(r"(?<=\d)\.(?=\d{3}(?:\D|$))","", regex=True).str.replace(",",".", regex=False)
    return pd.to_numeric(x, errors="coerce")

# --- header smarter ---
HEADER_KEYWORDS = (
    "date", "motif", "rédact", "redact", "trig", "nom", "prénom", "prenom",
    "personnel", "site", "retrait", "édition", "edition"
)

def choose_header_row(df: pd.DataFrame, scan=25) -> int:
    limit = min(len(df), scan)
    header_candidate_idx = None
    header_candidate_hits = -1
    for i in range(limit):
        row = df.iloc[i].astype(str)
        hits = 0
        for v in row:
            t = strip_accents_lower(v)
            if t and any(k in t for k in HEADER_KEYWORDS):
                hits += 1
        if hits >= 2 and hits > header_candidate_hits:
            header_candidate_hits = hits
            header_candidate_idx = i
    if header_candidate_idx is not None:
        return header_candidate_idx
    best, idx = -1, 0
    for i in range(limit):
        row = df.iloc[i].astype(str)
        non_empty = row.map(lambda x: x.strip()!="").sum()
        texty = row.map(lambda x: bool(re.search(r"[A-Za-zÀ-ÿ]", x))).sum()
        score = non_empty*2 + texty
        if score>best: best, idx = score, i
    return idx

def normalize_columns(header_vals):
    cols, seen = [], {}
    for v in header_vals:
        s = snake_id(v) if (v is not None and str(v).strip()!="") else "col"
        seen[s] = seen.get(s,0)+1
        cols.append(s if seen[s]==1 else f"{s}_{seen[s]}")
    return cols

def drop_empty_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(axis=1, how="all")
    blank_cols = [c for c in df.columns if df[c].dropna().astype(str).str.strip().eq("").all()]
    if blank_cols:
        df = df.drop(columns=blank_cols)
    return df

# --- inférence minimale robuste ---
MAP_TRUE  = {"true","vrai","oui","yes","1","y","o"}
MAP_FALSE = {"false","faux","non","no","0","n"}

# Regex SANS groupes capturants -> évite le warning pandas
DATE_TOKEN_RE = re.compile(
    r"(?:\d{4}[-/\.]\d{1,2}[-/\.]\d{1,2})|(?:\d{1,2}[-/\.]\d{1,2}[-/\.]\d{2,4})|"
    r"(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|janv|févr|fevr|avr|mai|juin|juil|sept|oct|nov|d[ée]c)",
    re.I
)
ISO_DATE_RE = re.compile(r"^\d{4}-\d{2}-\d{2}(?:[ T]\d{2}:\d{2}:\d{2})?$")
COLNAME_DATE_HINTS = ("date", "dt", "heure", "time")

def infer_col(s: pd.Series):
    s = s.copy()
    ss = s.astype("string").str.strip()
    ss = ss.where(~ss.fillna("").eq("0"), pd.NA)  # éviter 0 -> 1970-01-01

    # 0) ISO détecté -> on parse d'abord en ISO (dayfirst=False) pour éviter le warning
    iso_mask = ss.fillna("").str.match(ISO_DATE_RE)
    if iso_mask.mean() >= 0.5:
        dt_iso = pd.to_datetime(ss.where(iso_mask), errors="coerce", dayfirst=False)
        ok = dt_iso.notna().mean() if len(ss) else 0.0
        if ok >= 0.5:
            has_time = (dt_iso.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (
                dt_iso.dt.strftime("%Y-%m-%d %H:%M:%S") if has_time else dt_iso.dt.strftime("%Y-%m-%d"),
                "DATETIME" if has_time else "DATE"
            )

    # 1) Excel serial dates plausibles
    as_num = pd.to_numeric(ss, errors="coerce")
    frac_num = as_num.notna().mean() if len(ss) else 0.0
    if frac_num >= 0.9:
        mask = as_num.between(60, 2950000)  # >=60 = après 1900-03-01
        parsed = pd.to_datetime(as_num.where(mask), unit="D", origin="1899-12-30", errors="coerce")
        ok = parsed.notna().mean() if len(ss) else 0.0
        if ok >= 0.7:
            has_time = (parsed.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (
                parsed.dt.strftime("%Y-%m-%d %H:%M:%S") if has_time else parsed.dt.strftime("%Y-%m-%d"),
                "DATETIME" if has_time else "DATE"
            )

    # 2) tokens de date ou nom de colonne “datey”
    colname_hint = any(h in strip_accents_lower(s.name or "") for h in COLNAME_DATE_HINTS)
    date_token_ratio = ss.fillna("").str.contains(DATE_TOKEN_RE, na=False).mean()
    looks_datey = colname_hint or (date_token_ratio >= 0.30)
    if looks_datey:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="Could not infer format.*", category=UserWarning)
            dt1 = pd.to_datetime(ss, errors="coerce", dayfirst=True,  cache=True)
            dt2 = pd.to_datetime(ss, errors="coerce", dayfirst=False, cache=True)
        dt = dt2 if dt2.notna().sum() > dt1.notna().sum() else dt1
        ok = dt.notna().mean() if len(ss) else 0.0
        # seuil plus bas si le NOM de colonne suggère une date (ex: 'date_de_retrait')
        ok_thresh = 0.50 if colname_hint else 0.70
        if ok >= ok_thresh:
            has_time = (dt.dt.time.astype(str) != "00:00:00").mean() > 0.2
            return (
                pd.to_datetime(dt).dt.strftime("%Y-%m-%d %H:%M:%S") if has_time else pd.to_datetime(dt).dt.strftime("%Y-%m-%d"),
                "DATETIME" if has_time else "DATE"
            )

    # 3) bool strict
    mb = ss.str.lower().map(lambda x: True if x in MAP_TRUE else (False if x in MAP_FALSE else pd.NA))
    if (mb.notna().mean() if len(ss) else 0) >= 0.98:
        return mb.astype("boolean"), "BOOL"

    # 4) nombre
    nums = parse_number_like(ss)
    if (nums.notna().mean() if len(ss.dropna()) else 0) >= 0.85:
        nz = nums.dropna()
        if len(nz) and (np.mod(nz, 1) == 0).all():
            return nums.astype("Int64"), "INT"
        return nums.astype("Float64"), "FLOAT"

    # 5) texte
    return ss.astype("string"), "STRING"

def pg_type(tag: str) -> str:
    return {
        "DATE":"date", "DATETIME":"timestamp without time zone",
        "INT":"integer", "FLOAT":"double precision", "BOOL":"boolean"
    }.get(tag, "text")

# --- main ---
def main():
    xlsx = Path(FILE_PATH)
    if not xlsx.exists():
        print(f"[error] Fichier introuvable: {xlsx}"); return

    sheets = pd.read_excel(xlsx, sheet_name=None, engine="openpyxl", header=None)

    done = 0
    for name, raw in sheets.items():
        raw = raw.dropna(how="all", axis=0).dropna(how="all", axis=1)
        if raw.empty: continue

        h = choose_header_row(raw, scan=25)
        cols = normalize_columns(raw.iloc[h].tolist())
        df = raw.iloc[h+1:].copy()
        df.columns = cols

        df = df.dropna(how="all")
        df = drop_empty_columns(df)
        if df.empty: 
            continue

        schema = {}
        for c in df.columns:
            casted, tag = infer_col(df[c])
            df[c] = casted
            schema[c] = tag

        base = f"trigrammes.{snake_id(name)}"
        print(f"\n--- Feuille: {name} -> {base} ---")
        print(f"rows={len(df)} | cols={df.shape[1]}")
        print("Colonnes:", ", ".join(df.columns.astype(str)))
        print("\nSchema détecté:")
        for c in df.columns:
            print(f"  - {c}: {schema[c]}  ->  {pg_type(schema[c])}")

        if SHOW_SAMPLE:
            with pd.option_context("display.max_columns", 80, "display.width", 200):
                print("\nSample (top 8):")
                print(df.head(8))

        done += 1

    print(f"\nDone. Feuilles analysées: {done}")

if __name__ == "__main__":
    main()



--- Feuille: PDG -> trigrammes.pdg ---
rows=20 | cols=4
Colonnes: edition, date, motif, redacteur

Schema détecté:
  - edition: INT  ->  integer
  - date: DATE  ->  date
  - motif: STRING  ->  text
  - redacteur: STRING  ->  text

Sample (top 8):
    edition        date                                              motif    redacteur
9         1  2024-06-10                                           Création  S. BELMONTE
10        2  2024-09-16                           Ajout nouveaux arrivants    Y. RAGEOT
11        3  2024-10-21                                    Ajout couturier    Y. RAGEOT
12        4  2024-10-28  Mise à jour des dates d'entrée 
Ajout des habi...    Y. RAGEOT
13        5  2024-12-16  Ajout nouvelle arrivante - 1 personne - Feriel...  F.BOULHABEL
14        6  2024-12-18                              Départ de Yann RAGEOT  S. BELMONTE
15        7  2024-12-19  Ajout de nouveaux arrivants - 4 personnes - Fr...  F.BOULHABEL
16        8  2024-12-20  Ajout de nouvel arrivan